In [1]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain.agents import tool
load_dotenv()

Python-dotenv could not parse statement starting at line 10
Python-dotenv could not parse statement starting at line 11
Python-dotenv could not parse statement starting at line 12


True

In [2]:
@tool
def get_word_length(word):
    """Return length of word"""
    return len(word)+1

@tool
def get_capital(country):
    """Return capital of the coutry"""
    return "Delhi"



In [3]:
tools = [get_word_length,get_capital]

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ('system',  "provide the anwer of tool even it is incorrect"),
    ('user', "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),

])

In [13]:
llm = ChatGroq(model="llama3-70b-8192")

In [14]:
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm=llm, prompt= prompt, tools=tools)

In [15]:
from langchain.agents import AgentExecutor

executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [16]:
executor.invoke({'input': 'how many letter in  whatsup'})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'whatsup'}`


8The answer is 7.

> Finished chain.


{'input': 'how many letter in  whatsup', 'output': 'The answer is 7.'}

In [17]:
executor.invoke({'input': 'what is the capital of germany'})



> Entering new AgentExecutor chain...

Invoking: `get_capital` with `{'country': 'Germany'}`


DelhiIt seems like the tool made an error. Delhi is not the capital of Germany. The correct answer is Berlin.

> Finished chain.


{'input': 'what is the capital of germany',
 'output': 'It seems like the tool made an error. Delhi is not the capital of Germany. The correct answer is Berlin.'}

In [10]:
import rdflib
from rdflib import Graph, Literal, RDF, URIRef, Namespace, BNode
from rdflib.namespace import XSD, RDFS

class ModelDesignerAgent:
    def __init__(self):
        self.g = Graph()
        self.samm = Namespace("https://w3id.org/samm#")
        self.samm_c = Namespace("https://w3id.org/samm/characteristic#")
        self.ex = Namespace("http://example.org/")
        
        self.g.bind("samm", self.samm)
        self.g.bind("samm-c", self.samm_c)
        self.g.bind("ex", self.ex)

    def create_aspect(self, name, properties):
        aspect = self.ex[name]
        self.g.add((aspect, RDF.type, self.samm.Aspect))
        self.g.add((aspect, self.samm.preferredName, Literal(name, lang="en")))
        
        properties_node = BNode()
        self.g.add((aspect, self.samm.properties, properties_node))
        collection = rdflib.collection.Collection(self.g, properties_node)
        for prop in properties:
            collection.append(self.ex[prop])

    def create_property(self, name, description, characteristic):
        prop = self.ex[name]
        self.g.add((prop, RDF.type, self.samm.Property))
        self.g.add((prop, self.samm.preferredName, Literal(name, lang="en")))
        self.g.add((prop, self.samm.description, Literal(description, lang="en")))
        self.g.add((prop, self.samm.characteristic, self.ex[characteristic]))

    def create_characteristic(self, name, data_type):
        char = self.ex[name]
        self.g.add((char, RDF.type, self.samm.Characteristic))
        self.g.add((char, self.samm.preferredName, Literal(name, lang="en")))
        self.g.add((char, self.samm.dataType, XSD[data_type]))

    def process_user_input(self, user_input):
        for aspect_name, aspect_data in user_input.items():
            self.create_aspect(aspect_name, aspect_data['properties'].keys())
            
            for prop_name, prop_data in aspect_data['properties'].items():
                self.create_property(prop_name, prop_data['description'], prop_data['characteristic'])
                
                self.create_characteristic(prop_data['characteristic'], prop_data['dataType'])

    def get_samm_model(self):
        return self.g.serialize(format="turtle")

# Example usage
agent = ModelDesignerAgent()

user_input = {
    "Customer": {
        "properties": {
            "customerId": {
                "description": "Unique identifier for each customer",
                "characteristic": "CustomerIdCharacteristic",
                "dataType": "string"
            },
            "firstName": {
                "description": "Customer's first name",
                "characteristic": "NameCharacteristic",
                "dataType": "string"
            },
            "lastName": {
                "description": "Customer's last name",
                "characteristic": "NameCharacteristic",
                "dataType": "string"
            }
        }
    }
}

agent.process_user_input(user_input)
samm_model = agent.get_samm_model()
print(samm_model)

@prefix ex: <http://example.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix samm: <https://w3id.org/samm#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:Customer a samm:Aspect ;
    samm:preferredName "Customer"@en ;
    samm:properties ( ex:customerId ex:firstName ex:lastName ) .

ex:CustomerIdCharacteristic a samm:Characteristic ;
    samm:dataType xsd:string ;
    samm:preferredName "CustomerIdCharacteristic"@en .

ex:customerId a samm:Property ;
    samm:characteristic ex:CustomerIdCharacteristic ;
    samm:description "Unique identifier for each customer"@en ;
    samm:preferredName "customerId"@en .

ex:firstName a samm:Property ;
    samm:characteristic ex:NameCharacteristic ;
    samm:description "Customer's first name"@en ;
    samm:preferredName "firstName"@en .

ex:lastName a samm:Property ;
    samm:characteristic ex:NameCharacteristic ;
    samm:description "Customer's last name"@en ;
    samm:preferredName "lastName"@en .

ex:NameCh

In [11]:
!pip install rdflib